In [1]:
import os
from scipy.io import wavfile
import numpy as np
from IPython.display import Audio
import matplotlib.pyplot as plt

In [ ]:
def get_data_dict(datafolder='../../data/archive/recordings'):
    file_names = os.listdir(datafolder)

    data_dict = {}
    for file in file_names:
        key = file.split('.')[0]
        mat = wavfile.read(datafolder + '/{}'.format(file))[1].astype(np.float32)
        data_dict[key] = mat
    return data_dict

In [ ]:
dd = get_data_dict()
print(dd)

In [ ]:
file_names = os.listdir('../../data/archive/recordings_train')
max = 0
lengths = []
for file in file_names:
    mat_len = len(wavfile.read('../../data/archive/recordings_train' + '/{}'.format(file))[1].astype(np.float32))
    lengths.append(mat_len)
    if mat_len > max:
        max = mat_len
        
print(max)
plt.plot(lengths)

In [ ]:
file_names = os.listdir('../../data/archive/recordings_test')
max = 0
lengths = []

for file in file_names:
    mat_len = len(wavfile.read('../../data/archive/recordings_test' + '/{}'.format(file))[1].astype(np.float32))
    lengths.append(mat_len)
    if mat_len > max:
        max = mat_len
        
print(max)
plt.plot(lengths)

In [2]:
Audio(wavfile.read('../../data/archive/recordings_test' + '/0_jackson_0.wav')[1].astype(np.float32), rate=8000)

In [3]:
jackson = wavfile.read('../../data/archive/recordings_test' + '/0_jackson_0.wav')[1].astype(np.float32)
print(len(jackson))
if len(jackson) < 8000:
    jackson_padded = np.append(jackson, np.array([0.0 for i in range(8000 - len(jackson))]))
else:
    jackson_padded = jackson[0:8000]
print(jackson_padded.shape)
Audio(jackson_padded, rate=8000)

5148
(8000,)


In [ ]:
def get_data_dict(data):
    data_dict = {}
    for line in data:
        print(line)
        if "[" in line:
            key = line.split()[0]
            mat = []
        elif "]" in line:
            line = line.split(']')[0]
            mat.append([float(x) for x in line.split()])
            data_dict[key]=np.array(mat)
        else:
            mat.append([float(x) for x in line.split()])
    return data_dict

with open('./train_1digit.feat') as f:
    test_data = get_data_dict(f.readlines())
my_dict = get_data_dict(test_data)
print(my_dict)

In [11]:
import librosa

librosa_mfcc_feature = librosa.feature.mfcc(y=jackson_padded[:-1].astype(np.float32), sr=8000, n_mfcc=39, n_fft=1024, win_length=int(0.025*8000), hop_length=int(0.01*8000))
print(librosa_mfcc_feature.shape)

(39, 100)
